In [3]:
import sqlite3, json, random

In [4]:
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print("Connection error:", e)
    finally:
        return conn

In [5]:
def create_table(conn, table, columns):
    try:
        sql = f'Drop table if exists {table};'
        c = conn.cursor()
        c.execute(sql)
        conn.commit()
    except Exception as e:
        print('Table drop error:',e)
        
    try:
        sql = f'Create table if not exists {table} ('
        for column in columns:
            sql += f'{column[0]} {column[1]},'
        sql = sql[:-1] # remove trailing comma
        sql += ');'
        c = conn.cursor()
        c.execute(sql)
        conn.commit()
        print('Table created:', table)
    except sqlite3.Error as e:
        print("Table creation error:", e)

In [6]:
def insert_records(conn, table, columns, values):
    try:
        c = conn.cursor()
        sql = f'INSERT INTO {table} VALUES '
        for valset in values:
            sql += "("
            for val in valset:
                if isinstance(val, str): 
                    val = val.replace("'","''") # escape any single quote entries
                sql+= f"'{val}',"
            sql = sql[:-1] # remove last comma
            sql += "),"
        sql = sql[:-1] + ';' # remove last comma
        c.execute(sql)
        conn.commit()
        print('Inserted',len(values),'values')
    except sqlite3.Error as e:
        print("Record Insertion error:", e)

In [7]:
with open('default_rolltables.json') as f:
    rolltables = json.load(f)

In [8]:
db = 'rolltables.db'
conn = create_connection(db)
for rolltable in rolltables:
    table = rolltable['table']
    columns = rolltable['columns']
    values = rolltable['values']
    create_table(conn, table, columns)
    insert_records(conn, table, columns, values)
conn.close()

Table created: PlayerRace
Inserted 20 values
Table created: PlayerPersonality
Inserted 20 values
Table created: PlayerClass
Inserted 12 values
Table created: Environment
Inserted 12 values
Table created: StartArea
Inserted 10 values
Table created: DungeonType
Inserted 10 values
Table created: Passage
Inserted 10 values
Table created: PassageWidth
Inserted 9 values
Table created: Door
Inserted 10 values
Table created: DoorBeyond
Inserted 5 values
Table created: Stair
Inserted 14 values
Table created: Exit
Inserted 5 values
Table created: ExitLarge
Inserted 7 values
Table created: ExitLocation
Inserted 4 values
Table created: Odor
Inserted 14 values
Table created: Noise
Inserted 58 values
Table created: Air
Inserted 11 values
Table created: Feature
Inserted 52 values
Table created: Furnish
Inserted 88 values
Table created: Furnish_Religous
Inserted 41 values
Table created: Furnish_Mage
Inserted 60 values
Table created: Furnish_Utensil
Inserted 77 values
Table created: Container_Contents


In [13]:
# decrement ID by 1
for table in rolltables:
    for row in table['values']:
        row[0] = row[0]-1

In [14]:
with open("rolltables_fix.json", "w") as write_file:
    json.dump(rolltables, write_file, indent=4)

## Radom Select testing

In [107]:
def simple_query(sql):
    conn = sqlite3.connect('rolltables.db')
    c = conn.cursor()
    c.execute(sql)
    results = c.fetchall()
    conn.close()
    return results

In [114]:
sql = 'Select value from DungeonType;'
for item in enumerate(simple_query(sql)):
    print(item[0], item[1][0])

0 General
1 Lair
2 Tomb
3 Temple/Shrine
4 Stonghold
5 Planar Gate
6 Mine
7 Maze
8 Treasure Vault
9 Death Trap


In [97]:
def roll_query(table, columns):
    
    roll = random.randint(1,rtable_rows[table])
    
    if isinstance(columns, str) or columns == '*':
        select_cols = columns
    else:   
        select_cols = ''
        for col in columns:
            select_cols += col + ','
        select_cols = select_cols[:-1]
    
    sql = f"Select {select_cols} from (Select rowid, *, sum(weight) over(Order by rowid) rt from {table}) where rt >= {roll} order by rt asc limit 1;"
    conn = sqlite3.connect('rolltables.db')
    c = conn.cursor()
    c.execute(sql)
    results = c.fetchone()
    conn.close()
    return results

### view

In [10]:
%%timeit
query()

577 µs ± 3.97 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### subquery

In [14]:
%%timeit
query()

571 µs ± 3.08 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Final?

In [18]:
%%timeit
query('PlayerRace',['value'],29)

577 µs ± 3.57 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [47]:
r = query('PlayerRace',['rowid','value','weight'],29)
print("id:",r[0])
print("value:",r[1])
print("weight:",r[2])

id: 2
value: Gnome
weight: 2


In [83]:
def get_table_metadata():
    sql = 'select tbl_name from sqlite_master;'
    conn = sqlite3.connect('rolltables.db')
    c = conn.cursor()
    c.execute(sql)
    results = c.fetchall()
    rtable_rows={}
    for table in results:
        tbl_name = table[0]
        sql = f"Select count(1) rows from {tbl_name};"
        c.execute(sql)
        cnt = c.fetchone()
        rtable_rows[tbl_name] = cnt[0]
    conn.close()
    return rtable_rows

In [72]:
%%timeit
conn = sqlite3.connect('rolltables.db')
conn.close()

254 µs ± 599 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### base metadata

In [75]:
%%timeit
r = get_table_metadata()

476 µs ± 2.29 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [86]:
%%timeit
r = get_table_metadata()

1.23 ms ± 4.72 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [84]:
rtable_rows = get_table_metadata()
rtable_rows

{'PlayerRace': 20,
 'PlayerPersonality': 20,
 'PlayerClass': 12,
 'Environment': 12,
 'StartArea': 10,
 'DungeonType': 10,
 'Passages': 10,
 'PassageWidth': 9,
 'Door': 10,
 'DoorBeyond': 5,
 'Stair': 14,
 'Exit': 5}

In [85]:
rtable_rows['PlayerRace']

20

In [106]:
roll_query('PlayerClass','*')

(12,
 11,
 'Paladin',
 1,
 'STR',
 'CHA',
 'Light armor, medium armor, shields, simple weapons, martial weapons',
 12)

In [69]:
r = get_table_metadata()
roll_tables={}
for table in r:
    tbl_name = table[0]
    sql = f"Select count(1) rows from {tbl_name};"
    

Select count(1) rows from PlayerRace ;
Select count(1) rows from PlayerPersonality ;
Select count(1) rows from PlayerClass ;
Select count(1) rows from Environment ;
Select count(1) rows from StartArea ;
Select count(1) rows from DungeonType ;
Select count(1) rows from Passages ;
Select count(1) rows from PassageWidth ;
Select count(1) rows from Door ;
Select count(1) rows from DoorBeyond ;
Select count(1) rows from Stair ;
Select count(1) rows from Exit ;


In [188]:
r =''
i = 0
while r != 'Goliath' or i >10000:
    i+=1
    results = query('PlayerRace')
    r = results[1]
print(i)

8402


## Random select but in python

In [24]:
import pandas as pd
from random import randint

In [26]:
db = 'rolltables.db'
conn = create_connection(db)
sql = 'Select value, weight from exit;'
df = pd.read_sql(sql, conn)
roll = randint(1,len(df))
print(roll)
df['weight'].cumsum() <= roll

1


0    True
1    True
2    True
3    True
4    True
Name: weight, dtype: bool

## List of tables

In [ ]:
single_value_table_names = [
    'PlayerRace'
    ,'PlayerPersonality' 
    ,'PassageWidth'
    ,'Door'
    ,'Stair'
    ,'Environment'
    ,'ChamberExits'
    ,'ChamberExitsLarge'
    ,'ChamberExitLocation'
    ,'ChamberState'
    ,'ChamberPurpose'
    ,'ChamberPurposeDeath'
    ,'ChamberPurposeLair'
    ,'ChamberPurposeMaze'
    ,'ChamberPurposeMine'
    ,'ChamberPurposeTreasure'
    ,'ChamberPurposeTomb'
    ,'ChamberPurposeStrong'
    ,'ChamberPurposeTemple'
    ,'ChamberPurposeGate'
    ,'ChamberPurpose'
    ,'ChamberHazard'
    ,'ChamberObstacle'
    ,'TrapSeverity'
    ,'TrapTrigger'
    ,'TrapEffect'
    ,'TrickObject'
    ,'TrickEffect'
    ,'MonsterMotivation'
    
]

In [1]:
Multi_value_tables = [
    {'PlayerClass': ['Value','PrimaryStat','SecondaryStat','Proficencies']},
    {'Background': ['Value','Skills','Languages','Tools','Source','Page']},
    {'StartArea': ['Value','Passages','Doors']},
    {'Passage': ['Value','Chamber','Stair','Door','Passage']},
    {'DoorBeyond': ['Value','Chamber','Stair']},
    {'ChamberSize': ['Value','Large']},
    {'ExitType': ['Value','Door']},
    {'ChamberContents': ['Value','Type','Level']},
    {'Monster': ['Name','Environment','Size','Type','Tags','Alignment','Challenge','XP','Source','Page']},
]

## Weighted simulation testing

In [15]:
import random

In [153]:
def one_round(dic):
    winner = ["None",0]
    for k, v in dic.items():
        val = random.random()*v
        if val > winner[1]:
            winner[0] = k
            winner[1] = val
    return winner[0]

def simulation(rounds=100):
    i=1
    roll_table = {
        "5ft":2,
        "10ft":10,
        "20ft":2,
        "30ft":2,
        "40a":1,
        "40b":1,
        "40c":1,
        "40d":1,
        }
    winner_list = dict.fromkeys(roll_table, 0)

    for i in range(rounds):
        winner_list[one_round(roll_table)]+=1
    print(winner_list)

In [154]:
for i in range(20):
    simulation()

{'5ft': 7, '10ft': 82, '20ft': 5, '30ft': 6, '40a': 0, '40b': 0, '40c': 0, '40d': 0}
{'5ft': 6, '10ft': 85, '20ft': 4, '30ft': 4, '40a': 0, '40b': 1, '40c': 0, '40d': 0}
{'5ft': 8, '10ft': 85, '20ft': 3, '30ft': 4, '40a': 0, '40b': 0, '40c': 0, '40d': 0}
{'5ft': 3, '10ft': 87, '20ft': 2, '30ft': 7, '40a': 0, '40b': 1, '40c': 0, '40d': 0}
{'5ft': 4, '10ft': 89, '20ft': 6, '30ft': 1, '40a': 0, '40b': 0, '40c': 0, '40d': 0}
{'5ft': 2, '10ft': 90, '20ft': 5, '30ft': 3, '40a': 0, '40b': 0, '40c': 0, '40d': 0}
{'5ft': 6, '10ft': 81, '20ft': 6, '30ft': 6, '40a': 0, '40b': 0, '40c': 1, '40d': 0}
{'5ft': 5, '10ft': 83, '20ft': 6, '30ft': 4, '40a': 1, '40b': 0, '40c': 0, '40d': 1}
{'5ft': 5, '10ft': 83, '20ft': 4, '30ft': 7, '40a': 1, '40b': 0, '40c': 0, '40d': 0}
{'5ft': 3, '10ft': 85, '20ft': 6, '30ft': 6, '40a': 0, '40b': 0, '40c': 0, '40d': 0}
{'5ft': 7, '10ft': 88, '20ft': 2, '30ft': 2, '40a': 1, '40b': 0, '40c': 0, '40d': 0}
{'5ft': 5, '10ft': 87, '20ft': 5, '30ft': 3, '40a': 0, '40b': 0, 

In [1]:
test = 'lck_something'
test.replace("lck",'txt')

'txt_something'

In [17]:
ls = ['thing1','thing2','thing3','thing4']
ls2 = ['thing2','thing4']
l3 = [x for x in ls if x not in ls2]
l3

['thing1', 'thing3']

In [12]:
test = 'btn_rrt_Furnish'
test[6]

't'